In [1]:
import shutil,sys,os,time
import numpy as np
refFile="../VariantFiltration/Branchiostoma.belcheri_v18h27.r3_ref_genome.softmasked.fa"
vcfFile="Extracted.vcf" #or "52_bbe_copy.vcf". But need to rewrite.
snpQD_File="../VariantFiltration/52_bbe_copy_SNP_filter.vcf"
indelQD_File="../VariantFiltration/52_bbe_copy_INDEL_filter.vcf"
snpNoQD_File="../VariantFiltration/NO_QD_SNP.vcf"
indelNoQD_File="../VariantFiltration/NO_QD_INDEL.vcf"

## 一、标记long（>4bp) 和complex，及其上下游10bp。

In [2]:
### 1. 导入文件，并形成四个list
with open(vcfFile,"r") as h:  #h = gzip.GzipFile("52_bbe_copy.vcf.gz") 
    i=0
    line_list=[]
    tmp_list=[]
    vcf_chrom=[]
    vcf_pos=[]
    ref=[]
    alt=[]
    line_list=h.readlines()[1:]
    for line in line_list:
        tmp_list=line.split()
        vcf_chrom.append(tmp_list[0])
        vcf_pos.append(tmp_list[1])
        ref.append((tmp_list[2]).split(","))
        alt.append((tmp_list[3]).split(","))
        i+=1
vcf_pos=[int(x) for x in vcf_pos]

In [3]:
### 2.把所有变异分为五类：普通SNP，普通INDEL，复合，长，带星号的，标记到新的一列“CnL”。
i=0
CnL=[]  # "0.1" for normal SNP and "0.2" for normal INDEL, "1.1" for complex, "1.2" for long, "1.3" for *. (既是complex又是long的，则优先判断为complex)
star=0
norm_snp=0
norm_indel=0
long=0
compx=0
while i<len(vcf_chrom):
    if "*" in alt[i]:
        CnL.append(1.3)
        star+=1
        
    #These are INDELs.
    elif len(max(ref[i]+alt[i], key=len, default='1'))>1:
        ##complex
        if "*" in alt[i+1] or (len(ref[i][0])<len(max(alt[i], key=len)) and len(min(alt[i],key=len))==1):
            CnL.append(1.1)
            compx+=1
        ##long
        elif len(max(ref[i]+alt[i], key=len, default='1'))>4:
            CnL.append(1.2)
            long+=1
        ##normal
        else:
            CnL.append(0.2)
            norm_indel+=1
        
#These are normal SNPs.
    else: 
        CnL.append(0.1)
        norm_snp+=1
    i+=1

In [4]:
### 3.往下删除10bp。第一层循环：根据当前位点是否为complex/long；第二层循环：根据其下游的位点是否在10bp内。
i=0
j=0
maxPos=0
newPos=0
while i<len(CnL):
    if CnL[i]==0.1 or CnL[i]==0.2 or CnL[i]==1.3:
        pass
    
    elif CnL[i]==1.1 or CnL[i]==1.2:    #Complex/Long INDEL deletion.
        maxPos=vcf_pos[i]+len(ref[i][0])-1
        j=i+1
        while CnL[j]==1.3:
            newPos=vcf_pos[j]+len(ref[j][0])-1
            if newPos>maxPos:
                maxPos=newPos    #用最远的那个
            
            j+=1
            
            if j>len(CnL)-1:
                break
        
        while vcf_chrom[i]==vcf_chrom[j] and (CnL[j]==0.1 or CnL[j]==0.2) and vcf_pos[j]<=maxPos+10:
            CnL[j]=1.4   # For the normal SNP or INDEL which is within 10 bp downstream of complex/long, we assign the number "1.4".
            j+=1
            if j>len(CnL)-1:
                break
    i+=1            

In [5]:
### 4.相同方式，倒着数10bp
i=len(CnL)-1
j=0
while i>0:
    if CnL[i]==0.1 or CnL[i]==0.2 or CnL[i]==1.3 or CnL[i]==1.4:
        pass
    elif CnL[i]==1.1 or CnL[i]==1.2:
        j=i-1
        while vcf_chrom[i]==vcf_chrom[j] and (CnL[j]==0.1 or CnL[j]==0.2 or CnL[j]==1.4) and vcf_pos[j]>=vcf_pos[i]-10:
            CnL[j]=1.5   # For the normal SNP or INDEL which is within 10 bp upstream of complex/long, we assign the number "1.5".
            j-=1
            if j<0:
                break    #防止编号j溢出label
    i-=1

In [9]:
### 5. 第一步的统计
print("STEP 1 STATISTICS:\ntotal variants: ",len(vcf_pos),", normal SNP: ",norm_snp,", normal INDEL: ",norm_indel,", Complex Indel: ",compx,", long INDEL: ",long,", star: ",star)

kept=0
down10=0
up10=0
for snp in CnL:
    if snp==0.1 or snp==0.2:
        kept+=1
    elif snp==1.4:
        down10+=1
    elif snp==1.5:
        up10+=1
        
print("In step 1, ",len(CnL)-kept," variants (including",down10+up10,"are broadened to 10bp around), are marked as 'removed' ",kept," marked as kept.\n")

STEP 1 STATISTICS:
total variants:  39555599 , normal SNP:  26746475 , normal INDEL:  3661658 , Complex Indel:  2516666 , long INDEL:  2249629 , star:  4381171
In step 1,  17524292  variants (including 8376826 are broadened to 10bp around), are marked as 'removed'  22031307  marked as kept.



## 二、标记高密度位点

In [6]:
#之前是相当于每10bp取一个SNP，现在是相当于把高密度（>10bp）的区域都删了（留下第一个）。
#另起一列标签“HighDens”，高密度区域写成“1”，其它则为“0”。

HighDens=[]
for i in range(0,len(vcf_pos)):
    HighDens.append(0)

i=0
j=i+1

while i<len(vcf_pos)-2:
    if CnL[i]==0.1 or CnL[i]==0.2 or HighDens[i]==1:
        while CnL[j]==1.1 or CnL[j]==1.2 or CnL[j]==1.3 or CnL[j]==1.4 or CnL[j]==1.5:    ##Skip which are already marked as removed.
            j+=1
            if j>=len(vcf_pos)-2:
                break
        if vcf_chrom[j]==vcf_chrom[i] and vcf_pos[j]<vcf_pos[i]+len(ref[i][0])+9:
            HighDens[j]=1
    else:
        pass
    i=j
    j=i+1
    if i>=len(vcf_pos)-2:
        break

In [15]:
### 第二步的统计
dense=0
for i in range(0,len(vcf_pos)):
    if HighDens[i]==1:
        dense+=1
print("STEP 2 STATISTICS:\nIn step 2,",dense,'variants are within high-density regions.\n')

STEP 2 STATISTICS:
In step 2, 12823311 variants are within high-density regions.



## 三、标记测序质量低的位点

In [7]:
### 一、读取GATK生成的INDEL_filter.vcf、SNP_filter.vcf、NO_QD_INDEL.vcf、NO_QD_SNP.vcf四个文件：
with open(snpQD_File,"r") as h:
    count=0
    line_list=[]
    tmp_list=[]
    snp_chrom=[]
    snp_pos=[]
    snpQD=[]
    line_list=h.readlines()
    for line in line_list:
        if not line.startswith('#'):
            tmp_list=line.split()
            snp_chrom.append(tmp_list[0])
            snp_pos.append(tmp_list[1])
            snpQD.append(tmp_list[6])
            count+=1
snp_pos=[int(x) for x in snp_pos]

with open(indelQD_File,"r") as h:
    count=0
    line_list=[]
    tmp_list=[]
    indel_chrom=[]
    indel_pos=[]
    indelQD=[]
    line_list=h.readlines()
    for line in line_list:
        if not line.startswith('#'):
            tmp_list=line.split()
            indel_chrom.append(tmp_list[0])
            indel_pos.append(tmp_list[1])
            indelQD.append(tmp_list[6])
            count+=1
indel_pos=[int(x) for x in indel_pos]

with open(snpNoQD_File,"r") as h:
    count=0
    line_list=[]
    tmp_list=[]
    noQD_snp_chrom=[]
    noQD_snp_pos=[]
    line_list=h.readlines()
    for line in line_list:
        if not line.startswith('#'):
            tmp_list=line.split()
            noQD_snp_chrom.append(tmp_list[0])
            noQD_snp_pos.append(tmp_list[1])
            count+=1
noQD_snp_pos=[int(x) for x in noQD_snp_pos]

with open(indelNoQD_File,"r") as h:
    count=0
    line_list=[]
    tmp_list=[]
    noQD_indel_chrom=[]
    noQD_indel_pos=[]
    line_list=h.readlines()
    for line in line_list:
        if not line.startswith('#'):
            tmp_list=line.split()
            noQD_indel_chrom.append(tmp_list[0])
            noQD_indel_pos.append(tmp_list[1])
            count+=1
noQD_indel_pos=[int(x) for x in noQD_indel_pos]

In [8]:
### 将这几个文件的信息转化到一列标记

#### "1" for "Filter" SNP; "2" for "Filter" INDEL;
#### "0.1" for 'PASS' and with-QD-value SNP; "0.2" for 'PASS' and with-OD-value INDEL;
#### "3" for "PASS" but without-QD-value SNP; "4" for "PASS" but without-QD-value INDEL;
#### "99" for variants undetected by GATK.

QD=[]
for p in range(len(vcf_pos)):
    QD.append(99)

i=0    
for j in range(len(snp_pos)):
    while not (vcf_chrom[i]==snp_chrom[j] and vcf_pos[i]==snp_pos[j]): 
        i+=1
    if snpQD[j]=="PASS":
        QD[i]=0.1
    elif snpQD[j]=="Filter":
        QD[i]=1
    i+=1

i=0
for j in range(len(indel_pos)):
    while not (vcf_chrom[i]==indel_chrom[j] and vcf_pos[i]==indel_pos[j]): 
        i+=1
    if indelQD[j]=="PASS":
        QD[i]=0.2
    elif indelQD[j]=="Filter":
        QD[i]=2
    i+=1

i=0    
for j in range(len(noQD_indel_pos)):
    while not (vcf_chrom[i]==noQD_indel_chrom[j] and vcf_pos[i]==noQD_indel_pos[j]): 
        i+=1
    QD[i]=4
    i+=1
    
i=0    
for j in range(len(noQD_snp_pos)):
    while not (vcf_chrom[i]==noQD_snp_chrom[j] and vcf_pos[i]==noQD_snp_pos[j]): 
        i+=1
    QD[i]=3
    i+=1 

In [ ]:
### 第三步的统计：
print("STEP 3 STATISTICS:")
print("Counts of SNP defined by GATK: ",count)
print("Counts of indel defined by GATK: ",count)
print("Counts of SNP without QD-value: ",count)
print("Counts of INDEL without QD-value: ",count)

indel_filter=0
for QD in indelQD:
    if QD=="Filter":
        indel_filter+=1
print("Of all the indels, %s are labeled as 'PASS', while %s are labeled as 'Filter'."%(len(indelQD)-indel_filter,indel_filter))
snp_filter=0
for QD in snpQD:
    if QD=="Filter":
        snp_filter+=1
print("Of all the SNPs, %s are labeled as 'PASS', while %s are labeled as 'Filter'."%(len(snpQD)-snp_filter,snp_filter))

passSNP=passINDEL=yi=er=san=si=na=0
for qd in QD:
    if qd==0.1:
        passSNP+=1
    elif qd==0.2:
        passINDEL+=1
    elif qd==1:
        yi+=1
    elif qd==2:
        er+=1
    elif qd==3:
        san+=1
    elif qd==4:
        si+=1
    else:
        na+=1
print("%s SNP left, %s indel left; %s SNP labeled 'filter', %s indel labeled 'filter'; %s SNP pass \nbut no QD, %s indel pass but no QD; %s variants cannot be detected by GATK."%(passSNP,passINDEL,yi,er,san,si,na))

## 四、标记低重复区域（LCR）

In [9]:
#把fasta中的小写位点写成两个list
with open(refFile,"r") as f:
    chrom=[]
    pos=[]
    linelist=f.read().splitlines()
    ####用i数scaffold，j数行（每换一个scaffold归零一次），z数位点（每换一行归零一次）。
    i=0
    while i<len(linelist):
        j=1
        while not linelist[i+j].startswith('>'):
            for z in range(len(linelist[i+j])):
                if linelist[i+j][z].isupper():
                    pass
                else:
                    chrom.append(linelist[i][1:])
                    pos.append(z+1+(j-1)*50)
            j+=1
            if i+j>=len(linelist):
                break
        i+=j
z=list(zip(chrom,pos))

In [10]:
#求fasta的小写位点与vcf所有位点的交集
z_vcf=list(zip(vcf_chrom,vcf_pos))
intersect=list(set(z_vcf)&set(z))
sorted_intersect=sorted(intersect,key=lambda x:(x[0],x[1]))

#写入lcr的标记
lcr=[]
for i in range(len(vcf_pos)):
    lcr.append(0)
i=0    
for j in range(len(sorted_intersect)):
    while not z_vcf[i]==sorted_intersect[j]: 
        i+=1
    lcr[i]=1
    i+=1

In [18]:
### 第四步的统计
print("STEP 4 STATISTICS:\nlcr in fasta: ",len(pos))
count=0
for i in lcr:
    if i==1:
        count+=1
print("lcr in vcf: ",count,'\n')

STEP 4 STATISTICS:
lcr in fasta:  114764956
lcr in vcf:  11442773 



## 五、汇总

In [15]:
count=0
indel=0
snp=0
for i in range(len(vcf_pos)):
    if (CnL[i]==0.1 or CnL[i]==0.2) and HighDens[i]==0 and (QD[i]==0.1 or QD[i]==0.2) and lcr[i]==0:
        if len(max(ref[i]+alt[i], key=len, default='1'))>1:
            indel+=1
        else:
            snp+=1
        count+=1
print("After four steps, %s(%.f%%) left.%s(%.f%%) are SNPs; %s(%.f%%) are INDELs."%(count,count*100/len(vcf_pos),snp,snp*100/count,indel,indel*100/count))

After four steps, 5617000(14%) left.5057213(90%) are SNPs; 559787(10%) are INDELs.


In [27]:
for i in range(80,120):
    print(z_vcf[i],CnL[i],HighDens[i],QD[i],lcr[i])

('scaffold1', 1641) 1.3 0 2 1
('scaffold1', 1642) 1.3 0 99 1
('scaffold1', 1649) 1.3 0 2 1
('scaffold1', 1657) 1.3 0 1 1
('scaffold1', 1677) 1.2 0 0.2 1
('scaffold1', 1679) 1.5 0 1 0
('scaffold1', 1680) 1.5 0 0.2 0
('scaffold1', 1681) 1.1 0 99 0
('scaffold1', 1682) 1.5 0 1 0
('scaffold1', 1683) 1.1 0 99 0
('scaffold1', 1684) 1.4 0 1 0
('scaffold1', 1685) 1.4 0 1 0
('scaffold1', 1686) 1.4 0 0.1 0
('scaffold1', 1712) 0.1 0 0.1 0
('scaffold1', 1763) 0.1 0 0.1 0
('scaffold1', 1777) 0.1 0 0.1 0
('scaffold1', 1847) 0.1 0 1 0
('scaffold1', 1967) 0.1 0 1 0
('scaffold1', 2028) 0.1 0 1 0
('scaffold1', 2073) 1.2 0 0.2 0
('scaffold1', 2102) 0.1 0 0.1 0
('scaffold1', 2124) 0.1 0 1 0
('scaffold1', 2205) 1.2 0 0.2 0
('scaffold1', 2452) 0.1 0 1 1
('scaffold1', 2460) 0.1 1 1 1
('scaffold1', 2473) 0.1 0 0.1 1
('scaffold1', 2480) 0.2 1 0.2 1
('scaffold1', 2499) 0.2 0 0.2 1
('scaffold1', 2504) 0.1 1 0.1 1
('scaffold1', 2512) 1.5 0 0.2 0
('scaffold1', 2516) 1.5 0 0.1 0
('scaffold1', 2522) 1.2 0 0.2 0
('sca

## 六、输出到新文件

In [ ]:
####软删除(***还没跑过***)
with open("52_bbe_copy.vcf","r") as f:
    line_list=f.readlines()
    with open("techFilt_all.vcf","w") as g:
        g.write(line_list[0])
        tmp=[]
        for i in range(len(line_list)-1):
            tmp=line_list[i+1].append(CnL[i])
            tmp=tmp.append(HighDens[i])
            tmp=tmp.append(QD[i])
            tmp=tmp.append(lcr[i])
            g.write(line_list[i]+"\n")
        g.close()

In [16]:
####只保留删除后的
with open("52_bbe_copy.vcf","r") as f:
    line_list=f.readlines()[1:]
    with open("techFilt_left.vcf","w") as g:
        for i in range(len(line_list)):
            if (CnL[i]==0.1 or CnL[i]==0.2) and HighDens[i]==0 and (QD[i]==0.1 or QD[i]==0.2) and lcr[i]==0:
                g.write(line_list[i])
    g.close()